# Password Extraction
This is a script to extract password out of natural language inputs and in particular foursquare comments. It has been tested for passwords in the English language, it is not language dependent but it will be less efficient to process multi-lingual input.

It uses the occurences of words located before and after the password themselves to guess which word has the highest probability to be a password in the sentence.

In [1]:
import pandas as pd
import numpy as np
from collections import namedtuple
from sklearn import cross_validation
from sklearn.grid_search import RandomizedSearchCV
from sklearn.base import BaseEstimator
from sklearn.metrics import classification_report
import re
import pprint
import sys
import multiprocessing

In [2]:
TEST_SET_PERCENT = 0.3
BUDGET = 10**4
DATA_FILE = 'password_tips_english.csv'
EOL_TAG = '_eol_'
BOL_TAG = '_bol_'

In [3]:
class PasswordEstimator(BaseEstimator):
    """This find the word with the highest probability to be a password within the list of words

    Args:
        word_dicts_before: this is the dictionary of words found before the password ordered by 
            their distance, for example:
        [
            // words found at distand d = 1 from the password
            {
                ':': 1,
                'password': 0.4,
                'coffee': 0.1,
                'saxophone': 0.000001
            },
            // words found at distand d = 2 from the password
            {
                ':': 0.3,
                'password': 1,
                'wifi': 0.7
                'saxophone': 0.0000001
            }
        ]
        word_dicts_after: this is the dictionary of words found after the password ordered by 
            their distance
        before_cutoff: number of words to take into account before the passwords
        after_cutoff: number of words to take into account after the passwords
        before_exponential_factor: the exponentional factor that will give the ponderation of
            words before the password
        after_exponential_factor: the exponentional factor that will give the ponderation of
            words after the password
        after_factor: the factor to which words after the password account in comparison with
            words before the password
        min_password_length: the number of character minimum for a word to be considered as a
            password
        margin_cutoff: we can look not only at the frequency of the word at that distance of the
            password but also see the match we get if we move the word to the left or right
        margin_factor: the ponderation of the match on the margin of the word found
        eol_factor: the ponderation to give to newline
    """

    def __init__(
            self, before_cutoff=10, after_cutoff=10, before_exponential_factor=1,
            after_exponential_factor=1, after_factor=0.5, min_password_length=4, margin_cutoff=10,
            margin_factor=0.05, margin_exponential_factor=1, eol_factor=0.6, bol_factor=0.05):
        self.before_cutoff = before_cutoff
        self.after_cutoff = after_cutoff
        self.before_exponential_factor = before_exponential_factor
        self.after_exponential_factor = after_exponential_factor
        self.after_factor = after_factor
        self.min_password_length = min_password_length
        self.margin_cutoff = margin_cutoff
        self.margin_factor = margin_factor
        self.margin_exponential_factor = margin_exponential_factor
        self.eol_factor = eol_factor
        self.bol_factor = bol_factor

## Training
The training set is used to build dictionaries of words that occurs at a given position before and after the password.
- The password is identified in the tip
- The tip is split into the part precending and succeeding the password

Example:
```
    tip = 'The password is FNASDIYZXC.'
    password = 'FNASDIYZXC'
    tip_before, tip_after = get_tip_before_after_password(tip, password)
    print(tip_before) # 'The password is '
    print(tip_after) # '.'
```

In [4]:
def get_tip_before_after_password(self, tip, password):
    """Split the tip in the string before and after the password

    Args:
        tip: the tip containing the password. Example: 'The password is FNASDIYZXC'
        password: the known password in the tip. Example: 'FNASDIYZXC'

    Returns:
        A tuple containing the string before and the string after the password
        Example: ('The password is ', '.')
    """
    index_password = tip.index(password)
    return tip[:index_password], tip[index_password + len(password):]

PasswordEstimator.get_tip_before_after_password = get_tip_before_after_password

- Words are extracted for both parts and ordered by distance to the password.  Each tip is split into constituent words with the following considerations:
    - The ponctuation is considered as their own word.
    - The quotes are ignored.
    - Other ASCII characters are considered as part of other words if not separated by spaces.

The beginning and end of line is added to the list.
Example:
```
    words_before = get_words('The password is ', reverse=True, bol=True)
    words_after = get_words('.', eol=True)
    print(words_before) # ['is', 'password' 'The', '_bol_']
    print(words_after) # ['.', '_eol_']
```

In [5]:
def get_words(self, tip, reverse=False, bol=False, eol=False):
    """Split the string into words base on a regex

    Args:
        tip (string): the tip to be split. Example: 'The password is '
        reverse (bool): reverse the words if necessary
        bol (bool): contains beginning of line
        eol (bool): contains end of line

    Returns:
        The list of words
        Example: ['is', 'password' 'The', '_bol_']
    """
    # We consider the ponctuation as words on their own
    # Quotes are ignored
    words_case_sensitive = []
    if bol:
        words_case_sensitive += [BOL_TAG]
    words_case_sensitive += re.findall(r"[\w\/\\#$%@^_+*<=>&-]+|[.,!?;:]", tip)
    if eol:
        words_case_sensitive += [EOL_TAG]

    words_case_insensitive = [s.lower() for s in words_case_sensitive]

    if reverse:
        return list(reversed(words_case_sensitive)), list(reversed(words_case_insensitive))

    return words_case_sensitive, words_case_insensitive
PasswordEstimator.get_words = get_words

- The words are added to the dictionary corresponding to their position with respect to the password. 
- The value in each dictionary are normalized with respect to the greatest occurence found

Example of dictionary:
```
words_before_password = [
    # words found at distand d = 1 from the password
    {
        ':': 1,
        'password': 0.4,
        'coffee': 0.1,
        'saxophone': 0.000001,
        ...
    },
    # words found at distand d = 2 from the password
    {
        ':': 0.3,
        'password': 1,
        'wifi': 0.7
        'saxophone': 0.0000001,
        ...
    },
    ...
]

words_after_password = [
    # words found at distand d = 1 from the password
    {
        '_eol_': 1,
        'wifi': 0.4,
        ...
    },
    ...
```

In [6]:
def add_words(self, words, word_dicts, max_words):
    """Add words found in the training set to the word dictionary

    Args:
        words: the list of words to add order by their distance to the password
        word_dicts: the relevant word dictionaries to add to
        max_words: the number of words we want to add

    Returns:
        The filled word dictionaries
    """
    for i, word in enumerate(words):
        if i >= max_words:
            break

        if word not in word_dicts[i]:
            word_dicts[i][word] = 0
        word_dicts[i][word] += 1

    return word_dicts

PasswordEstimator.add_words = add_words

In [7]:
def normalize(self, word_dicts):
    """Normalize the words found

    Args:
        word_dicts: the word dictionaries to be normalized

    Returns:
        The normalized word dictionaries
    """
    for i, word_dict in enumerate(word_dicts):
        max_val = max(word_dict.values())
        word_dicts[i] = {k: v / max_val for k, v in word_dict.items()}
    return word_dicts

PasswordEstimator.normalize = normalize

We have now put the whole training together

In [8]:
def fit(self, X, y):
    """Train the model

    Args:
        X: the training set containing the tips
        y: the list of passwords matching the tips
    """
    self.word_dicts_before = [{} for _ in range(0, self.before_cutoff)]
    self.word_dicts_after = [{} for _ in range(0, self.after_cutoff)]

    for i, tip in X.iteritems():
        try:
            tip_before, tip_after = self.get_tip_before_after_password(tip, y[i])
        except ValueError:
            continue

        _, words_before = self.get_words(tip_before, reverse=True, bol=True)
        _, words_after = self.get_words(tip_after, eol=True)
        self.word_dicts_before = self.add_words(
            words_before, self.word_dicts_before, self.before_cutoff)
        self.word_dicts_after = self.add_words(
            words_after, self.word_dicts_after, self.after_cutoff)

    self.word_dicts_before = self.normalize(self.word_dicts_before)
    self.word_dicts_after = self.normalize(self.word_dicts_after)
    return self

PasswordEstimator.fit = fit

## Predicting

The prediction is done by evaluating for every word the likeliness of it being a password by looking at the words preceding and succeeding it.

- The tip is split into its consituent words
```
    tip_words = self.get_words(tip, eol=True, bol=True)
    print(tip_words) # ['_bol_', 'The', 'password', 'is', 'FNASDIYZXC', '.', '_eol_']
```
- For each guess we split the words into two array of words that come before and after the password
```
    guessing = words[i] # 'FNASDIYZXC'
    words_before = list(reversed(words[:i])) # ['is', 'password' 'The', '_bol_']
    words_after = words[i + 1:] # ['.', '_eol_']
```
- We calculate the score of each array, their score is ponderated by a factor given as a parameter
```
    score_guess = score_before + score_after * self.factor_after
```
- For every word in the array we add the normalized occurence of the word in the dictionary at that position. We multiply this socre by a factor depending on the distance of the work to the password.
```
    scoring_word = 'password'
    position = 1 # index is 1 so distance is 2
    score_word = words_before_password[position][scoring_word] * self.ponderate(self.exponential_factor, position)
```

In [9]:
def ponderate(self, exponential_factor, distance):
    """Ponderate the word score using an inverse exponentional

    Args:
        exponential_factor: the exponentional factor
        distance: the x axis

    Returns:
        The ponderator
    """
    return np.exp(-1 * exponential_factor * distance)

PasswordEstimator.ponderate = ponderate

In [10]:
def get_score(self, words, word_dicts, max_words, exponential_factor, idx):
    """Calculate the likeliness that a potential word is a password based on the list of words
        after or before the password

    Args:
        words: the list of words to consider
        word_dicts: the relevant word dictionary
        max_words: the number of words to consider
        exponential_factor: the exponentional factor for the word ponderation based on
            their distance to the password

    Returns:
        The score of this side of the password
    """
    score = 0
    for i in range(0, max_words):
        word = None
        word_dict = word_dicts[i]
        if i < len(words):
            word = words[i]

        if word not in word_dict:
            continue

        word_score = word_dict[word]

        # margin right
        left_border = i + 1
        right_border = min(i + 1 + self.margin_cutoff, max_words)
        score_right_margin = self.get_score_margin(left_border, right_border, word, i, word_dicts, idx)
        word_score += score_right_margin * self.margin_factor

        # margin left
        left_border = max(i - 1 - self.margin_cutoff, 0)
        right_border = i - 1
        score_left_margin = self.get_score_margin(left_border, right_border, word, i, word_dicts, idx)
        word_score += score_left_margin * self.margin_factor

        word_score *= self.ponderate(exponential_factor, i)
        if word == EOL_TAG:
            word_score *= self.eol_factor
        if word == BOL_TAG:
            word_score *= self.bol_factor

        score += word_score
    return score
    
PasswordEstimator.get_score = get_score

- We also add the score given by the occurence of that  word in the dictionary on both sides of the given position. This gives us the chance to give value to a word that is very frequent at a different position with respect to the password then it is in the evaluated sentence.
```
    scoring_word = 'password'
    position = 1
    position_shift = -1 # we're looking at the 'left' site of the word 
    score_word += words_before_password[position + position_shift][scoring_word] \
                    * factor(position) \
                    * margin_factor(position_shift)
```

In [11]:
def get_score_margin(self, left_border, right_border, word, index, word_dicts, idx):
    """Calculate the score given by the left or right margin of the word

    Args:
        left_border: the left side of the interval we are looking at
        right_border: the right side of the interval we are looking at
        word: the word we are calculating the score for
        index: the distance from the password the word is at
        word_dicts: the dictionary of word counts

    Returns:
        The score given by this margin
    """
    score = 0
    for j in range(left_border, right_border):
        if word in word_dicts[j]:
            distance = np.absolute(j - index) - 1
            score += word_dicts[j][word] * self.ponderate(
                self.margin_exponential_factor, distance)
    return score

PasswordEstimator.get_score_margin = get_score_margin

We can now put the prediting function together

In [12]:
def predict(self, X):
    """Predict a list of password for the given tips

    Args:
        X: the list of tips

    Returns:
        The list of predicted passwords
    """

    y = []
    for idx, tip in X.iteritems():
        words_case_sensitive, words = self.get_words(tip, eol=True, bol=True)

        best_score = 0
        best_guess = ''

        for i in range(0, len(words)):
            guessing = words_case_sensitive[i]

            if guessing == EOL_TAG or guessing == BOL_TAG:
                continue

            if len(guessing) < self.min_password_length:
                continue

            words_before = list(reversed(words[:i]))
            words_after = words[i + 1:]
            score_before = self.get_score(
                words_before, self.word_dicts_before, self.before_cutoff,
                self.before_exponential_factor, idx)
            score_after = self.get_score(
                words_after, self.word_dicts_after, self.after_cutoff,
                self.after_exponential_factor, idx)
            score = score_before + score_after * self.after_factor

            if score > best_score:
                best_guess = guessing
                best_score = score

        y.append(best_guess)

    return pd.Series(y, index=X.index)

PasswordEstimator.predict = predict

## Main

Load the data and extract annotated data

In [13]:
df = pd.read_csv('data/' + DATA_FILE)
data = df[(df.password.notnull()) & (df.done == '1')]

Passwords = namedtuple('Passwords', 'data target')
pwds = Passwords(data=data.tip, target=data.password)

Split into training and test set

In [14]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    pwds.data, pwds.target, test_size=TEST_SET_PERCENT, random_state=0)

Set the parameter space

In [15]:
parameters = {
    'before_cutoff': list(range(0, 6)),
    'after_cutoff': list(range(0, 6)),
    'before_exponential_factor': np.logspace(-1, 1, 10),
    'after_exponential_factor': np.logspace(-1, 1, 10),
    'after_factor': np.logspace(-2, 1, 20),
    'min_password_length': list(range(1, 10)),
    'margin_cutoff': list(range(0, 3)),
    'margin_factor': np.logspace(-2, 0, 10),
    'margin_exponential_factor': np.logspace(-1, 1, 10),
    'eol_factor': np.logspace(-2, 0, 10),
    'bol_factor': np.logspace(-2, 0, 10)
}

Use random search and k-fold cross validation, the time this takes depends on the budget chosen, increase the budget to find more accurate parameters

In [16]:
clf = RandomizedSearchCV(
    PasswordEstimator(), parameters, cv=2, scoring='accuracy',
    n_jobs=multiprocessing.cpu_count() - 1, n_iter=BUDGET)

clf = clf.fit(X_train, y_train)

In [17]:
pprint.pprint(clf.best_params_)

{'after_cutoff': 2,
 'after_exponential_factor': 2.1544346900318834,
 'after_factor': 0.54555947811685168,
 'before_cutoff': 3,
 'before_exponential_factor': 0.774263682681127,
 'bol_factor': 0.59948425031894093,
 'eol_factor': 1.0,
 'margin_cutoff': 1,
 'margin_exponential_factor': 3.5938136638046259,
 'margin_factor': 0.016681005372000592,
 'min_password_length': 4}


Get the final score on the test set

In [18]:
test_score = clf.score(X_test, y_test)
print(test_score)

0.917261055635
